In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

# Feature Engineering

In [19]:
df = pd.read_csv("/Users/eva/Documents/Study/Y1S2/DMT/assignment2/cleaned_training_set_VU_DM.csv")
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,bool_visitor_hist
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
3,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,27348,2,4.0,...,0.0,0.0,0.0,-1.0,0.0,5.0,0,0.0,0,0
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0


In [20]:
df_test = pd.read_csv("/Users/eva/Documents/Study/Y1S2/DMT/assignment2/cleaned_test_set_VU_DM.csv")

## Find columns with numeric values so that we can caluculate mean, std, median per prop_id


In [21]:
# 上下链接两个df
combined_df = pd.concat([df, df_test], ignore_index=False)

In [5]:
bool_flag_cols = [col for col in df.columns if 'bool' in col.lower() or 'flag' in col.lower()]

# transform the columns to boolean
for col in bool_flag_cols:
    df[col] = df[col].astype(bool)


In [6]:
print(df['date_time'].dtypes)
# transform it into datetime
df['date_time'] = pd.to_datetime(df['date_time'])


object


In [7]:
# print columns end with rate and env
rate_cols = df.columns[df.columns.str.endswith('rate')]
inv_cols = [col for col in df.columns if 'inv' in col.lower()]
print(rate_cols)
print(inv_cols)

Index(['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate',
       'comp6_rate', 'comp7_rate', 'comp8_rate'],
      dtype='object')
['comp1_inv', 'comp2_inv', 'comp3_inv', 'comp4_inv', 'comp5_inv', 'comp6_inv', 'comp7_inv', 'comp8_inv']


In [8]:
numeric_cols = [col for col in df.select_dtypes(include=['int64', 'float64']).columns if 'id' not in col and col not in rate_cols and col not in inv_cols]
print(numeric_cols)
print(len(numeric_cols))

['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_query_affinity_score', 'orig_destination_distance', 'comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff', 'gross_bookings_usd']
25


In [22]:
features = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_query_affinity_score', 'orig_destination_distance', 'comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff']

# 计算每个组合的均值、中位数和标准差
grouped = combined_df.groupby(['prop_id', 'srch_id', 'visitor_location_country_id'])
stats = grouped[features].agg(['mean', 'median', 'std'])

original_train_len = len(df)
original_test_len = len(df_test)
for feature in features:
    for id_col in ['prop_id', 'srch_id', 'visitor_location_country_id']:
        for stat in ['mean', 'median', 'std']:
            column_name = f'{feature}_{id_col}_{stat}'
            combined_df[column_name] = grouped[feature].transform(stat)

/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_90832/2390605579.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[column_name] = grouped[feature].transform(stat)
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_90832/2390605579.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[column_name] = grouped[feature].transform(stat)
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_90832/2390605579.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

In [24]:
df = combined_df.iloc[:original_train_len]
df_test = combined_df.iloc[original_train_len:original_train_len + original_test_len]

In [9]:
# calculate mean, std, median for each prop_id
grouped = df.groupby('prop_id')[numeric_cols]
avg_df = grouped.mean().reset_index().rename(columns=lambda x: x + '_prop_id_mean' if x != 'prop_id' else x)
std_df = grouped.std().reset_index().rename(columns=lambda x: x + '_prop_id_std' if x != 'prop_id' else x)
median_df = grouped.median().reset_index().rename(columns=lambda x: x + '_prop_id_median' if x != 'prop_id' else x)

# merge the dataframes
df = df.merge(avg_df, on='prop_id')
df = df.merge(std_df, on='prop_id')
df = df.merge(median_df, on='prop_id')


In [10]:
# calculate mean, standard deviation, median for each site_id
grouped = df.groupby('srch_id')[numeric_cols]
avg_df = grouped.mean().reset_index().rename(columns=lambda x: x + '_srch_id_mean' if x != 'srch_id' else x)
std_df = grouped.std().reset_index().rename(columns=lambda x: x + '_srch_id_std' if x != 'srch_id' else x)
median_df = grouped.median().reset_index().rename(columns=lambda x: x + '_srch_id_median' if x != 'srch_id' else x)

# merge the dataframes
df = df.merge(avg_df, on='srch_id')
df = df.merge(std_df, on='srch_id')
df = df.merge(median_df, on='srch_id')

In [11]:
# calculate mean, standard deviation, median for each destination_id
grouped = df.groupby('visitor_location_country_id')[numeric_cols]
avg_df = grouped.mean().reset_index().rename(columns=lambda x: x + '_visitor_location_country_id_mean' if x != 'visitor_location_country_id' else x)
std_df = grouped.std().reset_index().rename(columns=lambda x: x + '_visitor_location_country_id_std' if x != 'visitor_location_country_id' else x)
median_df = grouped.median().reset_index().rename(columns=lambda x: x + '_visitor_location_country_id_median' if x != 'visitor_location_country_id' else x)

# merge the dataframes
df = df.merge(avg_df, on='visitor_location_country_id')
df = df.merge(std_df, on='visitor_location_country_id')
df = df.merge(median_df, on='visitor_location_country_id')


: 

In [25]:
#to_csv
df.to_csv('/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_set_VU_DM.csv', index=False)

In [27]:
df_test.to_csv('/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_test_set_VU_DM.csv', index=False)

In [26]:
# truncate into chunks
# Load the large CSV file
file_path = '/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_set_VU_DM.csv'
data = pd.read_csv(file_path)

# Define the size of each chunk
chunk_size = 500000  # This number can change 

# Split the CSV into chunks
for i in range(0, len(data), chunk_size):
    chunk = data.iloc[i:i + chunk_size]
    chunk.to_csv(f'/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_chunk_{i//chunk_size}.csv', index=False)

In [7]:
df = pd.read_csv(f'/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_chunk_0.csv')

In [23]:
#print number of df columns
print(len(combined_df.columns))


271
